# 2. Weather data cleaning

In this section we will focus on the weather data set. 

Import of packages:

In [4]:
import pandas as pd
import numpy as np

Import of data set:

In [5]:
df_weather = pd.read_csv('data\weather_hourly_boston.csv')

## 2.1 Getting a first overview of the weather data

In [6]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43848 entries, 0 to 43847
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_time  43354 non-null  object 
 1   max_temp   43354 non-null  float64
 2   min_temp   43354 non-null  float64
 3   precip     43356 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.3+ MB


In [7]:
df_weather.columns

Index(['date_time', 'max_temp', 'min_temp', 'precip'], dtype='object')

In [8]:
df_weather.head(5)

,date_time,max_temp,min_temp,precip
0,2015-01-02 01:00:00,-1.1,-1.1,0.0
1,2015-01-02 02:00:00,-1.1,-1.1,0.0
2,2015-01-02 03:00:00,-0.6,-0.6,0.0
3,2015-01-02 04:00:00,-0.6,-0.6,0.0
4,2015-01-02 05:00:00,-0.6,-0.6,0.0


In [9]:
df_weather.tail(5)

,date_time,max_temp,min_temp,precip
43843,2020-01-01 20:00:00,5.0,5.0,0.0
43844,2020-01-01 21:00:00,4.4,4.4,0.0
43845,2020-01-01 22:00:00,4.4,4.4,0.0
43846,2020-01-01 23:00:00,3.9,3.9,0.0
43847,2020-01-02 00:00:00,3.3,3.3,0.0


## 2.2 Identifying missing or wrong values and duplicates

In [10]:
df_weather.isnull().values.sum()

1974

In [11]:
df_weather.duplicated().sum()

1070

Apparently there are some null values and also duplicates, which need to be handled later on.

## 2.3 Clean bad data

### 2.3.1 Remove data from other years
The dataset contains information about the weather from 2015 - 2020. As only data of 2017 are needed, the data can be limited to this specific year. 

In [12]:
df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])

df_weather_2017 = df_weather[(df_weather['date_time'] >= "2017-01-01") & (df_weather['date_time'] < "2018-01-01")]

In [13]:
df_weather_2017 = df_weather_2017.sort_values('date_time', ascending = True)
df_weather_2017

,date_time,max_temp,min_temp,precip
17543,2017-01-01 00:00:00,3.9,3.9,0.0
17520,2017-01-01 01:00:00,4.4,4.4,0.0
17521,2017-01-01 02:00:00,5.0,5.0,1.0
17522,2017-01-01 03:00:00,5.0,5.0,1.0
17523,2017-01-01 04:00:00,5.0,4.4,1.0
...,...,...,...,...
26298,2017-12-31 19:00:00,-11.1,-11.1,0.0
26299,2017-12-31 20:00:00,-10.6,-10.6,0.0
26300,2017-12-31 21:00:00,-11.1,-11.1,0.0
26301,2017-12-31 22:00:00,-11.7,-11.7,0.0


To display information about every hour in 2017 the weather data should contain 365 * 24 = 8760 rows. As the filtered dataframe only has 8690 entries, there are probably 70 hours missing.

### 2.3.2 Handle null values

In [14]:
df_weather_2017.isnull().values.sum()

0

Fortunatley no NA values are in the dataset anymore. 

### 2.3.3 Handle duplicates

In [15]:
df_weather_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8690 entries, 17543 to 26302
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_time  8690 non-null   datetime64[ns]
 1   max_temp   8690 non-null   float64       
 2   min_temp   8690 non-null   float64       
 3   precip     8690 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 339.5 KB


In [16]:
dup = len(df_weather_2017[df_weather_2017.duplicated(subset = 'date_time', keep = False)])
dup_records = len(df_weather_2017[df_weather_2017.duplicated(keep = 'first')])
dup_date_times = len(df_weather_2017) - len(df_weather_2017['date_time'].unique()) - dup_records

print("Apparently there are %d duplicate date times, which after closer examination are divided into %d pairs of duplicate date times and %d pairs of 'actual' duplicated records." % (dup, dup_date_times, dup_records))

Apparently there are 1028 duplicate date times, which after closer examination are divided into 382 pairs of duplicate date times and 132 pairs of 'actual' duplicated records.


In [17]:
# Filtering out all 'actual' duplicate records of our DataFrame
df_weather_2017 = df_weather_2017.drop_duplicates(keep = 'first')
len(df_weather_2017)#.duplicated().sum()

8558

So far we have already removed the 132 "actual" duplicated records out of our data set by keeping only the first one. Now lets check for the 382 pairs of duplicates. 

In [18]:
# Select all duplicate date times with different precip values in the data set
# Pseudo: Select all duplicate records of 'date_time' with different values for the "precip" column (A.precip != B.precip)
    # select * 
    # from A join (select * from A group by A.date_time having count(*) > 1) B on A.date_time = B.date_time
    # where A.precip != B.precip
    # order by A.date_time;
df_weather_precip_dup = df_weather_2017[df_weather_2017.duplicated(subset = 'date_time', keep = False)].groupby(['date_time']).filter(lambda x: len(x['precip'].unique()) > 1)
len(df_weather_precip_dup)

156

156 of 382 data rows have a different value for the "precip" column of the data frame. 


In [19]:
rainDays = len(df_weather_2017[df_weather_2017["precip"]==1])
sunDays = len(df_weather_2017[df_weather_2017["precip"]==0])

print("There are only %d rain days compared to the %d sun days in the data set. For simplicitiy, we remove the duplicated records where precipitation is '1' and since it is only a small number it would not make a huge impact." %(rainDays, sunDays))

There are only 835 rain days compared to the 7723 sun days in the data set. For simplicitiy, we remove the duplicated records where precipitation is '1' and since it is only a small number it would not make a huge impact.


In [20]:
# Remove the duplicated date times with precipitation = 1 of the data set
df_weather_precip_sub = df_weather_precip_dup[(df_weather_precip_dup['precip'] == 1)]
df_weather_2017 = df_weather_2017.drop(index = df_weather_precip_sub.index)
df_weather_2017

,date_time,max_temp,min_temp,precip
17543,2017-01-01 00:00:00,3.9,3.9,0.0
17520,2017-01-01 01:00:00,4.4,4.4,0.0
17521,2017-01-01 02:00:00,5.0,5.0,1.0
17522,2017-01-01 03:00:00,5.0,5.0,1.0
17523,2017-01-01 04:00:00,5.0,4.4,1.0
...,...,...,...,...
26298,2017-12-31 19:00:00,-11.1,-11.1,0.0
26299,2017-12-31 20:00:00,-10.6,-10.6,0.0
26300,2017-12-31 21:00:00,-11.1,-11.1,0.0
26301,2017-12-31 22:00:00,-11.7,-11.7,0.0


Now we are removing the duplicate values in the column "date_time".

In [21]:
df_weather_2017 = df_weather_2017.drop_duplicates(subset='date_time', keep='first')
len(df_weather_2017)

8176

## 2.4 Fill the missing data
After removing and handling the duplicate date times of our data set, we need to determine which date times are missing and fill in those missing records, since we only have 8176 out of 8760 records.

### 2.4.1 Add missing hours

In [22]:
# Setting the date_time as DateTimeIndex of the Dataframe to "fill in" the missing date times
df_weather_2017.set_index('date_time', inplace = True)
df_weather_2017

,max_temp,min_temp,precip
date_time,,,
2017-01-01 00:00:00,3.9,3.9,0.0
2017-01-01 01:00:00,4.4,4.4,0.0
2017-01-01 02:00:00,5.0,5.0,1.0
2017-01-01 03:00:00,5.0,5.0,1.0
2017-01-01 04:00:00,5.0,4.4,1.0
...,...,...,...
2017-12-31 19:00:00,-11.1,-11.1,0.0
2017-12-31 20:00:00,-10.6,-10.6,0.0
2017-12-31 21:00:00,-11.1,-11.1,0.0


In [23]:
# "Filling in" the missing date times to interpolate the missing values
df_weather_modified = df_weather_2017.asfreq(freq='1H')
df_weather_modified

,max_temp,min_temp,precip
date_time,,,
2017-01-01 00:00:00,3.9,3.9,0.0
2017-01-01 01:00:00,4.4,4.4,0.0
2017-01-01 02:00:00,5.0,5.0,1.0
2017-01-01 03:00:00,5.0,5.0,1.0
2017-01-01 04:00:00,5.0,4.4,1.0
...,...,...,...
2017-12-31 19:00:00,-11.1,-11.1,0.0
2017-12-31 20:00:00,-10.6,-10.6,0.0
2017-12-31 21:00:00,-11.1,-11.1,0.0


### 2.4.2 Interpolate missing values
The steady course of the temperature allows us to replace the missing values by the mean values of the surrounding values.

In [24]:
# Interpolating the missing values
df_weather_modified = df_weather_modified.interpolate(method = 'linear')
df_weather_modified.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2017-01-01 00:00:00 to 2017-12-31 23:00:00
Freq: H
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   max_temp  8760 non-null   float64
 1   min_temp  8760 non-null   float64
 2   precip    8760 non-null   float64
dtypes: float64(3)
memory usage: 273.8 KB


In [25]:
df_weather_modified

,max_temp,min_temp,precip
date_time,,,
2017-01-01 00:00:00,3.9,3.9,0.0
2017-01-01 01:00:00,4.4,4.4,0.0
2017-01-01 02:00:00,5.0,5.0,1.0
2017-01-01 03:00:00,5.0,5.0,1.0
2017-01-01 04:00:00,5.0,4.4,1.0
...,...,...,...
2017-12-31 19:00:00,-11.1,-11.1,0.0
2017-12-31 20:00:00,-10.6,-10.6,0.0
2017-12-31 21:00:00,-11.1,-11.1,0.0


## 2.5 Export cleaned data set
In order to reuse the cleaned wheather data for further analysis, we export that dataset into a new csv file.

In [26]:
df_weather_modified['date_time'] = df_weather_modified.index
df_weather_modified.reset_index(drop = True)

,max_temp,min_temp,precip,date_time
0,3.9,3.9,0.0,2017-01-01 00:00:00
1,4.4,4.4,0.0,2017-01-01 01:00:00
2,5.0,5.0,1.0,2017-01-01 02:00:00
3,5.0,5.0,1.0,2017-01-01 03:00:00
4,5.0,4.4,1.0,2017-01-01 04:00:00
...,...,...,...,...
8755,-11.1,-11.1,0.0,2017-12-31 19:00:00
8756,-10.6,-10.6,0.0,2017-12-31 20:00:00
8757,-11.1,-11.1,0.0,2017-12-31 21:00:00
8758,-11.7,-11.7,0.0,2017-12-31 22:00:00


In [27]:
df_weather_modified.to_csv('weather_2017_cleaned.csv', index = False, header=True)